In [19]:
import numpy as np

class site:
    def __init__(self, position, properties):
        self.index = position
        for name,value in properties.items():
            setattr(self, name, value)
        

class Ising:
    def __init__(self, inL, inh0):
        self.L = inL
        self.h0 = inh0
        self.system = []
        self.system.append(site(0, {'Jleft':0, 'h':np.random.uniform(0,self.h0)}))
        for i in range(1,self.L):
            self.system.append(site(i, {'Jleft':np.random.uniform(0,1), 'h':np.random.uniform(0,self.h0)}))
    
    def printSystem(self):
        print('h' + str(0) + ' = ' + str(self.system[0].h))
        for i in range(1,self.L):
              print('J' + str(i-1) + ',' + str(i) + ' = ' + str(self.system[i].Jleft) + '\nh' + str(i) + ' = ' + str(self.system[i].h))
        print()
    
    def renormStep(self):
        bond = None
        Omega = 0
        OmegaPos = None
        for i in range(self.L):
            if Omega<self.system[i].Jleft:
                Omega = self.system[i].Jleft
                OmegaPos = i
                bond = True
            if Omega<self.system[i].h:
                Omega = self.system[i].h
                OmegaPos = i
                bond = False
        
        if bond:
            hright = self.system[OmegaPos].h
            hleft = self.system[OmegaPos-1].h
            del self.system[OmegaPos]
            self.system[OmegaPos-1].h = hright*hleft/Omega
        else:
            if OmegaPos<(self.L-1) and OmegaPos != 0:
                jleft = self.system[OmegaPos].Jleft
                jright = self.system[OmegaPos+1].Jleft
                del self.system[OmegaPos]
                self.system[OmegaPos].Jleft = jleft*jright/Omega
            else:
                if OmegaPos == 0:
                    del self.system[OmegaPos]
                    self.system[OmegaPos].Jleft = 0
                else:
                    del self.system[OmegaPos]
                
        
        self.L -= 1
        
        
        
tester = Ising(10,5)

tester.printSystem()
tester.renormStep()
tester.printSystem()

h0 = 3.533049069779657
J0,1 = 0.181453052701386
h1 = 1.5311284463565218
J1,2 = 0.0653936167328939
h2 = 2.486628091296239
J2,3 = 0.22977597092159696
h3 = 4.631310718992397
J3,4 = 0.14961746595486825
h4 = 0.8202075084712895
J4,5 = 0.4481033495877028
h5 = 3.607321877638367
J5,6 = 0.43408476874236146
h6 = 2.8160502919283203
J6,7 = 0.6582758886707952
h7 = 4.608558337851177
J7,8 = 0.9070513454498905
h8 = 4.812551782377634
J8,9 = 0.8813650351617399
h9 = 2.9875537650268975

h0 = 3.533049069779657
J0,1 = 0.181453052701386
h1 = 1.5311284463565218
J1,2 = 0.0653936167328939
h2 = 2.486628091296239
J2,3 = 0.22977597092159696
h3 = 4.631310718992397
J3,4 = 0.14961746595486825
h4 = 0.8202075084712895
J4,5 = 0.4481033495877028
h5 = 3.607321877638367
J5,6 = 0.43408476874236146
h6 = 2.8160502919283203
J6,7 = 0.6582758886707952
h7 = 4.608558337851177
J7,8 = 0.1661163094189051
h8 = 2.9875537650268975

